**Project Goal**: Write a Python function to calculate BMI from patient height and weight, then apply it to a list of patients from a mock relational database and to additionally categorize the BMI (underweight, normal, overweight, obese). It also has a script to update the data with a new patient info and this can be automated for larger patient datasets.

BMI is a number used to estimate a person’s body fat based on their weight and height. It’s widely used in healthcare to screen for weight categories that may lead to health problems.

🧮 How to Calculate BMI

Take the person's weight in kilograms (kg)

Take the person’s height in centimeters (cm)

Convert the height from cm to meters

Since 1 meter = 100 cm, divide the height by 100

Square the height in meters

Divide the weight by the squared height

BMI = (weight in kg) / (height in m)^2  = a number or ratio with no units

Below code uses SQL (sqlite3 for RAM access, Python, and Pandas)

In [ ]:
import pandas as pd
import sqlite3

# creates a new SQLite database connection
# The special string ":memory:" tells SQLite to create the database in RAM (memory) instead of on disk.
conn = sqlite3.connect (":memory:")

# creates a cursor object from the connection.
# execute SQL commands through the cursor.
cursor = conn.cursor()

# Create the Table in the relational (structured rows & cols) Database
cursor.execute (
    """
    CREATE TABLE bmi_records (
      id INTEGER PRIMARY KEY AUTOINCREMENT,
      firstName TEXT,
      lastName TEXT,
      weight_kg REAL,
      height_m REAL,
      bmi REAL
    )
    """
)

patients = {
    ("Ada", "Nguyen", 68.0, 1.65),
    ("Liam", "Johnson", 82.0, 1.75),
    ("Zoe", "Smith", 54.0, 1.60),
    ("Noah", "Lee", 95.0, 1.80),
    ("Maya", "Patel", 72.0, 1.70),
    ("Ethan", "Kim", 61.0, 1.68),
    ("Ava", "Brown", 48.5, 1.55),
    ("Oliver", "Garcia", 102.3, 1.90),
    ("Sophia", "Martinez", 63.0, 1.66),
    ("Jackson", "Chen", 76.2, 1.73)
}

for firstName, lastName, weight_kg, height_m in patients:
    bmi = weight_kg / (height_m ** 2) # assume appropriate units
    cursor.execute(
        """
        INSERT INTO bmi_records (firstName, lastName, weight_kg, height_m, bmi)
        VALUES (?, ?, ?, ?, ?)
        """,
    (firstName, lastName, weight_kg, height_m, bmi))

df = pd.read_sql_query("SELECT * FROM bmi_records", conn)
print(df)

   id firstName  lastName  weight_kg  height_m        bmi
0   1       Ada    Nguyen       68.0      1.65  24.977043
1   2      Liam   Johnson       82.0      1.75  26.775510
2   3       Zoe     Smith       54.0      1.60  21.093750
3   4      Noah       Lee       95.0      1.80  29.320988
4   5    Oliver    Garcia      102.3      1.90  28.337950
5   6    Sophia  Martinez       63.0      1.66  22.862534
6   7      Maya     Patel       72.0      1.70  24.913495
7   8       Ava     Brown       48.5      1.55  20.187305
8   9     Ethan       Kim       61.0      1.68  21.612812
9  10   Jackson      Chen       76.2      1.73  25.460256


In [36]:
# Delete duplicate Olivia Grayson, keeping the one with the lowest id
cursor.execute("""
    DELETE FROM bmi_records
    WHERE id NOT IN (
        SELECT MIN(id) FROM bmi_records
        GROUP BY firstName, lastName
    )
    AND (firstName = 'Olivia' AND lastName = 'Grayson')
""")

conn.commit()


In [42]:
df = pd.read_sql_query("SELECT * FROM bmi_records", conn)
print(df)

    id firstName  lastName  weight_kg  height_m        bmi
0    1       Ada    Nguyen       68.0      1.65  24.977043
1    2      Liam   Johnson       82.0      1.75  26.775510
2    3       Zoe     Smith       54.0      1.60  21.093750
3    4      Noah       Lee       95.0      1.80  29.320988
4    5    Oliver    Garcia      102.3      1.90  28.337950
5    6    Sophia  Martinez       63.0      1.66  22.862534
6    7      Maya     Patel       72.0      1.70  24.913495
7    8       Ava     Brown       48.5      1.55  20.187305
8    9     Ethan       Kim       61.0      1.68  21.612812
9   10   Jackson      Chen       76.2      1.73  25.460256
10  19    Olivia   Grayson       55.8      1.64  20.746579


In [ ]:
def print_bmi_report (firstname, lastname, bmi_value, bmi_category):
    """
    Prints a formatted BMI report for a given person.
    """
    print(f"{firstname, lastname} has a BMI of {bmi_value:.2f}, which falls in the {bmi_category} category.")

In [ ]:
def calculate_bmi(weight_kg, height_m):
    return weight_kg / (height_m ** 2)

In [ ]:
def categorize_bmi(bmi):
    if bmi < 18.5:
        return "Underweight"
    elif 18.5 <= bmi < 25:
        return "Normal"
    elif 25 <= bmi < 30:
        return "Overweight"
    else:
        return "Obese"